In [18]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow as tf
import json
import random
import pickle
# import curses
try:
    with open('intents.json', 'r') as file:
        data = json.load(file)
        
#         print(data["intents"])
        
except json.JSONDecodeError:
    print("Error: The file 'intents.json' contains invalid JSON data.")

try:
    with open("data.pickle", "rb") as f:
              words, labels, training, output = pickle.load(f)
    
except:
    words = []
    labels = []
    documents = []
    doc = []

    for intent in data["intents"]:
              for pattern in intent.get("patterns", []):
                  wrds = nltk.word_tokenize(pattern)
                  words.extend(wrds)
                  documents.append(wrds)
                  doc.append(intent["tag"])

              if intent["tag"] not in labels:
                  labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
#     print(words)

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, docs in enumerate(documents):
              bag = []

              wrds = [stemmer.stem(w) for w in docs]

              for w in words:
                  if w in wrds:
                      bag.append(1)
                  else:
                      bag.append(0)

              output_row = out_empty[:]
              output_row[labels.index(doc[x])] = 1

              training.append(bag)
              output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
              pickle.dump((words, labels, training, output),f)
    
              
# noinspection PyUnresolvedReferences
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 20)
net = tflearn.fully_connected(net, 20)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
try:
    model.load("Synergy")
except:
    model.fit(training, output, n_epoch=5000, batch_size=20, show_metric=True)
    model.save("Synergy")
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
              
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
              
    for wor in s_words:
        for i, w in enumerate(words):
                if w == wor:
                    bag[i] = 1
    return numpy.array(bag)
def chat():
    print("Lloyds bank!(type done to stop) ")
    while True:
        inp = input("You: ")
        if inp.lower() == "done":
            break
        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        for tg in data["intents"]:
            if tg["tag"] == tag:
                responses = tg["responses"]
        print(random.choice(responses))
              
chat()

INFO:tensorflow:Restoring parameters from C:\Users\HP G5\Chatbot\Synergy
Lloyds bank!(type done to stop) 
You: hello
Hello!
You: what accounts do you offer
If online,Just answer a few simple questions and enter information 
You: how can i open an account
What can I help you with?
You: what are the requirements for opening an account
At least 18years old
You: done
